In [3]:
import pandas as pd
from ast import literal_eval

In [4]:
from CarlaBEV.src.planning.planner import Planner

In [5]:
def append_to_scene(scene_id, actor, class_id):
    scene = []
    start, goal = actor
    scene.append([scene_id, class_id, start, goal, None, None])
    return scene

def compute_routes(scene_df):
    for idx, row in scene_df.iterrows():
        id = 0
        _, class_id, start, goal, _, _ = row
        if class_id == "pedestrians":
            id = 1
        planner = Planner(id)
        rx, ry = planner.find_global_path(start, goal, 1024)
        scene_df.astype({'rx': "object", 'ry': "object"}).dtypes
        scene_df.loc[idx, "rx"] = rx
        scene_df.loc[idx, "ry"] = ry
    return scene_df

def get_scene_df(scene_id, actors_dict):
    scene = []
    cols = ["scene_id", "class", "start", "goal", "rx", "ry"]
    for class_id in actors_dict.keys(): 
        for actors in actors_dict[class_id]:
            scene.extend(append_to_scene(scene_id, actors, class_id))

    scene_df = pd.DataFrame(scene, columns=cols)
    scene_df = compute_routes(scene_df)
    scene_df.to_csv(f"{scene_id}.csv")
    return scene_df

In [23]:
scene_id = "scene_1-1"

pedestrians = [
	[(8625, 4500), (8625, 1500)],
	[(8630, 2900), (8630, 1500)],
	[(8770, 6500), (8770, 1800)],
	[(8770, 1800), (8770, 6500)],
]

vehicles = [
	[(8730, 1800), (8730, 6500)],
	[(8730, 2300), (8730, 6500)],
	[(8650, 6500), (8650, 1500)],
	[(8650, 2900), (8650, 1500)],
]

scene_dict_11 = {
	"pedestrians": pedestrians,
	"vehicles": vehicles
}

scene_df_11 = get_scene_df(scene_id, scene_dict_11)

In [6]:
scene_id = "scene_1-2"

pedestrians = [
    [(8625, 4500), (8625, 1500)],
    [(8630, 2900), (8630, 1500)],
    [(8770, 6500), (8770, 1800)],
]

vehicles = [
    [(8730, 2300), (8730, 6500)],
    [(8650, 6500), (8650, 1500)],
]


scene_dict_12 = {
	"pedestrians": pedestrians,
	"vehicles": vehicles
}

scene_df_12 = get_scene_df(scene_id, scene_dict_12)

In [7]:
scene_id = "scene_1-3"
pedestrians = [
    [(8630, 2900), (8630, 1500)],
    [(8770, 1800), (8770, 6500)],
]


vehicles = [
    [(8730, 1800), (8730, 6500)],
    [(8730, 2300), (8730, 6500)],
]

scene_dict_13 = {
	"pedestrians": pedestrians,
	"vehicles": vehicles
}

scene_df_13 = get_scene_df(scene_id, scene_dict_13)

# LOAD SCENES

In [1]:
from CarlaBEV.src.scenes import SceneBuilder

In [8]:
scene_builder = SceneBuilder(["scene_1-1", "scene_1-2", "scene_1-3"], 128)